<a href="https://colab.research.google.com/github/inesfrsantos/MachineLearning/blob/main/Model_Attempts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Download the necessary packages
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from PIL import Image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from keras.utils import to_categorical

In [7]:
#Load the data
df = 'https://raw.githubusercontent.com/inesfrsantos/MachineLearning/main/Dataset/CleanedDataset/filtered_data.csv'
df = pd.read_csv(df)

In [8]:
X = df['image'].apply(lambda x: np.fromstring(x, dtype=np.uint8))

<ipython-input-8-098b9423becd>:1: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  X = df['image'].apply(lambda x: np.fromstring(x, dtype=np.uint8))


In [9]:
# Define a fixed image size
img_size = (64, 64)

# Resize images to the fixed size
X_resized = []
for img in X.values:
    img = Image.fromarray(img)
    img = img.resize(img_size)
    img = np.asarray(img)
    X_resized.append(img)

# Stack the resized images
X_stacked = np.stack(X_resized).reshape(-1, 64, 64, 1)


In [10]:
X_stacked.shape

(38645, 64, 64, 1)

In [14]:
#Split the data into train, test and validation
X_train, X_test, y_train, y_test = train_test_split(X_stacked, df['label'], test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

print("Number of training samples:", len(X_train))
print("Number of validation samples:", len(X_valid))
print("Number of testing samples:", len(X_test))


Number of training samples: 23187
Number of validation samples: 7729
Number of testing samples: 7729


In [15]:
# Enconde the labels
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_valid = le.transform(y_valid)
y_test = le.transform(y_test)
# Convert the labels to one-hot encoding
num_classes = 32
y_train = to_categorical(y_train, num_classes)
y_valid = to_categorical(y_valid, num_classes)
y_test = to_categorical(y_test, num_classes)


In [ ]:
import numpy as np
n_classes = len(np.unique(y_train))
print(n_classes)

2


In [16]:
y_train.shape

(23187, 32)

In [17]:
X_train.shape

(23187, 64, 64, 1)

# Simple self made CNN

In [ ]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# Define the model architecture
def create_model(num_filters=32, kernel_size=(3,3), activation='relu', learning_rate=0.01, batch_size=32):
    model = Sequential()
    model.add(Conv2D(num_filters, kernel_size=kernel_size, activation=activation, input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation=activation))
    model.add(Dense(num_classes, activation='softmax'))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Create a KerasClassifier for use with GridSearchCV
model = KerasClassifier(build_fn=create_model, epochs=10, verbose=1)

# Define the hyperparameters to search over
param_grid = {
    'num_filters': [32, 64],
    'kernel_size': [(3,3), (5,5)],
    'activation': ['relu', 'tanh'],
    'learning_rate': [0.001, 0.01],
    'batch_size': [32, 64]
}

# Define the GridSearchCV object
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

# Fit the GridSearchCV object on the training data
grid_result = grid.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_result.best_params_

# Train the final model using the best hyperparameters
final_model = create_model(**best_params)
final_model.fit(X_train, y_train, epochs=10, batch_size=best_params['batch_size'], validation_data=(X_valid, y_valid))


<ipython-input-14-abf74f495fa0>:20: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=10, verbose=1)


Epoch 1/10
484/484 [==============================] - 59s 118ms/step - loss: 10.7068 - accuracy: 0.0355
Epoch 2/10
484/484 [==============================] - 58s 121ms/step - loss: 3.4520 - accuracy: 0.0412
Epoch 3/10
484/484 [==============================] - 57s 118ms/step - loss: 3.4480 - accuracy: 0.0438
Epoch 4/10
484/484 [==============================] - 57s 118ms/step - loss: 3.4444 - accuracy: 0.0477
Epoch 5/10
484/484 [==============================] - 58s 119ms/step - loss: 3.4410 - accuracy: 0.0476
Epoch 6/10
484/484 [==============================] - 59s 122ms/step - loss: 3.4367 - accuracy: 0.0481
Epoch 7/10
484/484 [==============================] - 58s 120ms/step - loss: 3.4347 - accuracy: 0.0474
Epoch 8/10
484/484 [==============================] - 58s 119ms/step - loss: 3.4355 - accuracy: 0.0495
Epoch 9/10
484/484 [==============================] - 57s 119ms/step - loss: 3.4307 - accuracy: 0.0502
Epoch 10/10
242/242 [==============================] - 7s 28ms/step - lo

KeyboardInterrupt: ignored

In [ ]:
# Load packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import numpy as np

# Define the input shape
input_shape = (64, 64, 1)

# Initialize the model
model = Sequential()

# Add the first convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))

# Add the first max pooling layer
model.add(MaxPooling2D((2, 2)))

# Add the second convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))

# Add the second max pooling layer
model.add(MaxPooling2D((2, 2)))

# Add the flatten layer
model.add(Flatten())

# Add the dense layer
model.add(Dense(128, activation='relu'))

# Add the dropout layer
model.add(Dropout(0.5))

# Add the output layer
model.add(Dense(32, activation='softmax'))

def lr_decay(epoch):
    return 0.01 * 0.9 ** epoch

optimizer = Adam(learning_rate=0.01)

lr_scheduler = LearningRateScheduler(lr_decay)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_valid, y_valid))

# Evaluate the model on the test set
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/10
725/725 [==============================] - 264s 362ms/step - loss: 3.8335 - accuracy: 0.0435 - val_loss: 3.4588 - val_accuracy: 0.0454
Epoch 2/10
725/725 [==============================] - 253s 349ms/step - loss: 3.4528 - accuracy: 0.0438 - val_loss: 3.4556 - val_accuracy: 0.0454
Epoch 3/10
725/725 [==============================] - 251s 347ms/step - loss: 3.4530 - accuracy: 0.0445 - val_loss: 3.4566 - val_accuracy: 0.0454
Epoch 4/10
725/725 [==============================] - 253s 350ms/step - loss: 3.4532 - accuracy: 0.0443 - val_loss: 3.4584 - val_accuracy: 0.0454
Epoch 5/10
725/725 [==============================] - 250s 345ms/step - loss: 3.4531 - accuracy: 0.0436 - val_loss: 3.4580 - val_accuracy: 0.0454
Epoch 6/10
725/725 [==============================] - 249s 344ms/step - loss: 3.4529 - accuracy: 0.0444 - val_loss: 3.4585 - val_accuracy: 0.0454
Epoch 7/10
725/725 [==============================] - 250s 345ms/step - loss: 3.4530 - accuracy: 0.0435 - val_loss: 3.4575 -

# Complex Sef Made CNN

In [ ]:
# Add the first convolutional layer with 32 filters of size (3, 3), using 'same' padding
# Input shape is (64, 64, 1) for grayscale images, and ReLU activation function
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(64, 64, 1), activation='relu'))

# Add batch normalization to normalize the activations of the previous layer
model.add(BatchNormalization())

# Add the second convolutional layer with 32 filters of size (3, 3), using 'same' padding
# ReLU activation function is used
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))

# Add batch normalization to normalize the activations of the previous layer
model.add(BatchNormalization())

# Add max pooling layer with a pool size of (2, 2) to downsample the image
# Dropout of 0.25 is applied to randomly set 25% of the input units to 0 during training
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Add the third convolutional layer with 64 filters of size (3, 3), using 'same' padding
# ReLU activation function is used
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))

# Add batch normalization to normalize the activations of the previous layer
model.add(BatchNormalization())

# Add the fourth convolutional layer with 64 filters of size (3, 3), using 'same' padding
# ReLU activation function is used
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))

# Add batch normalization to normalize the activations of the previous layer
model.add(BatchNormalization())

# Add max pooling layer with a pool size of (2, 2) to downsample the image
# Dropout of 0.25 is applied to randomly set 25% of the input units to 0 during training
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Add the fifth convolutional layer with 128 filters of size (3, 3), using 'same' padding
# ReLU activation function is used
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))

# Add batch normalization to normalize the activations of the previous layer
model.add(BatchNormalization())

# Add the sixth convolutional layer with 128 filters of size (3, 3), using 'same' padding
# ReLU activation function is used
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))

# Add batch normalization to normalize the activations of the previous layer
model.add(BatchNormalization())

# Add max pooling layer with a pool size of (2, 2) to downsample the image
# Dropout of 0.25 is applied to randomly set 25% of the input units to 0 during training
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flatten the output from the previous layer to a 1D vector
model.add(Flatten())

# Add a fully connected dense layer with 512 units and ReLU activation function
model.add(Dense(512, activation='relu'))

# Add batch normalization to normalize the activations of the previous layer
model.add(BatchNormalization())

# Add Dropout o.5, meaning half of the neurons will be dropped
model.add(Dropout(0.5))

# Add the output layer with 28 units (one for each class) and softmax activation
model.add(Dense(32, activation='softmax'))


ValueError: ignored

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Load and preprocess data
X_train, y_train, X_val, y_val, X_test, y_test = load_and_preprocess_data()

# Define the model
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), padding='same', input_shape=(64, 64, 1), activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.25),
    layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(32, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train,
                    epochs=50


The architecture used for Arabic sign language image classification is effective because it includes more layers and parameters, allowing the model to learn more complex features from the images. The use of batch normalization helps to normalize the input, which can improve the convergence of the model during training. The addition of dropout layers helps to prevent overfitting by randomly dropping out nodes during training. The use of multiple convolutional and max pooling layers allows the model to learn hierarchical representations of the input images, capturing both local and global features. This architecture can handle more complex patterns and variations in the input data compared to a simpler model with only a few layers, like the simple CNN.

# GoogLeNet Inspired CNN

## Option 1

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, Flatten, Dense

# Define the input shape
input_shape = (64, 64, 1)

# Input layer
input_layer = Input(shape=input_shape)

# First convolutional block
conv1_1 = Conv2D(64, (1,1), activation='relu', padding='same')(input_layer)
conv1_2 = Conv2D(64, (3,3), activation='relu', padding='same')(conv1_1)
conv1_3 = Conv2D(192, (3,3), activation='relu', padding='same')(conv1_2)
pool1 = MaxPooling2D((3,3), strides=(2,2), padding='same')(conv1_3)

# Second convolutional block
conv2_1 = Conv2D(64, (1,1), activation='relu', padding='same')(pool1)
conv2_2 = Conv2D(128, (3,3), activation='relu', padding='same')(conv2_1)
conv2_3 = Conv2D(256, (3,3), activation='relu', padding='same')(conv2_2)
pool2 = MaxPooling2D((3,3), strides=(2,2), padding='same')(conv2_3)

# Third convolutional block with additional dropout and batch normalization
conv3_1 = Conv2D(128, (1,1), activation='relu', padding='same')(pool2)
conv3_2 = Conv2D(256, (3,3), activation='relu', padding='same')(conv3_1)
conv3_3 = Conv2D(512, (3,3), activation='relu', padding='same')(conv3_2)
pool3 = MaxPooling2D((3,3), strides=(2,2), padding='same')(conv3_3)
dropout1 = Dropout(0.4)(pool3)
batch_norm1 = BatchNormalization()(dropout1)

# Fourth convolutional block with additional dropout and batch normalization
conv4_1 = Conv2D(256, (1,1), activation='relu', padding='same')(batch_norm1)
conv4_2 = Conv2D(512, (3,3), activation='relu', padding='same')(conv4_1)
conv4_3 = Conv2D(1024, (3,3), activation='relu', padding='same')(conv4_2)
dropout2 = Dropout(0.4)(conv4_3)
batch_norm2 = BatchNormalization()(dropout2)

# Fifth convolutional block with additional dropout and batch normalization
conv5_1 = Conv2D(256, (1,1), activation='relu', padding='same')(batch_norm2)
conv5_2 = Conv2D(512, (3,3), activation='relu', padding='same')(conv5_1)
conv5_3 = Conv2D(1024, (3,3), activation='relu', padding='same')(conv5_2)
dropout3 = Dropout(0.4)(conv5_3)
batch_norm3 = BatchNormalization()(dropout3)

# Sixth convolutional block with additional dropout and batch normalization
conv6_1 = Conv2D(512, (1,1), activation='relu', padding='same')(concat)
conv6_2 = Conv2D(1024, (3,3), activation='relu', padding='same')(conv6_1)
dropout4 = Dropout(0.4)(conv6_2)
batch_norm4 = BatchNormalization()(dropout4)

# Seventh convolutional block with additional dropout and batch normalization
conv7_1 = Conv2D(256, (1,1), activation='relu', padding='same')(batch_norm4)
conv7_2 = Conv2D(512, (3,3), activation='relu', padding='same')(conv7_1)
conv7_3 = Conv2D(1024, (3,3), activation='relu', padding='same')(conv7_2)
dropout5 = Dropout(0.4)(conv7_3)
batch_norm5 = BatchNormalization()(dropout5)

# Eighth convolutional block with additional dropout and batch normalization
conv8_1 = Conv2D(256, (1,1), activation='relu', padding='same')(batch_norm5)
conv8_2 = Conv2D(512, (3,3), strides=(2,2), activation='relu', padding='same')(conv8_1)
conv8_3 = Conv2D(1024, (3,3), activation='relu', padding='same')(conv8_2)
dropout6 = Dropout(0.4)(conv8_3)
batch_norm6 = BatchNormalization()(dropout6)

# Ninth convolutional block with additional dropout and batch normalization
conv9_1 = Conv2D(512, (1,1), activation='relu', padding='same')(batch_norm6)
conv9_2 = Conv2D(1024, (3,3), activation='relu', padding='same')(conv9_1)
dropout7 = Dropout(0.4)(conv9_2)
batch_norm7 = BatchNormalization()(dropout7)

# Flatten the output of the ninth convolutional block
flatten = Flatten()(batch_norm7)

# Add two fully connected (dense) layers with additional dropout and batch normalization
dense1 = Dense(1024, activation='relu')(flatten)
dropout8 = Dropout(0.4)(dense1)
batch_norm8 = BatchNormalization()(dropout8)
dense2 = Dense(10, activation='softmax')(batch_norm8) # output layer with 10 classes

# Define the model with the input layer and the output layer
model = Model(inputs=input_layer, outputs=dense2)

# Print the model summary
model.summary()


The modifications made to the original GoogLeNet architecture are:

Using a grayscale input image (so only 1 channel instead of 3)
Changing the input image size to 64x64
Adding dropout and batch normalization layers after each of the third, fourth, fifth, sixth, seventh, eighth, and ninth convolutional blocks
Changing the number of output classes in the last dense layer to 10 (since we don't know what the specific classification task is, but it's common to have 10 classes for image classification problems)

## Option 2

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, concatenate, AveragePooling2D, Flatten, Dense, Dropout

# Define input shape
input_shape = (64, 64, 1)

# Define input layer
inputs = Input(shape=input_shape)

# Define GoogLeNet-like architecture
# First convolutional layer
conv1_7x7_s2 = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(inputs)

# Max pooling layer
pool1_3x3_s2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(conv1_7x7_s2)

# Second convolutional layer
conv2_3x3_reduce = Conv2D(64, (1, 1), padding='same', activation='relu')(pool1_3x3_s2)
conv2_3x3 = Conv2D(192, (3, 3), padding='same', activation='relu')(conv2_3x3_reduce)

# Max pooling layer
pool2_3x3_s2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(conv2_3x3)

# Inception module 1
inception_3a_1x1 = Conv2D(64, (1, 1), padding='same', activation='relu')(pool2_3x3_s2)

# These 3 convolutional layers are added in the modified GoogLeNet architecture.
conv3_3x3 = Conv2D(64, (3, 3), padding='same', activation='relu')(pool2_3x3_s2)
conv4_5x5 = Conv2D(32, (5, 5), padding='same', activation='relu')(pool2_3x3_s2)
conv5_1x1 = Conv2D(32, (1, 1), padding='same', activation='relu')(pool2_3x3_s2)

# Concatenate the output of the convolutional layers above
concatenate_1 = concatenate([inception_3a_1x1, conv3_3x3, conv4_5x5, conv5_1x1], axis=3)

# Inception module 2
inception_3b_1x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(concatenate_1)

# These 3 convolutional layers are added in the modified GoogLeNet architecture.
conv6_3x3 = Conv2D(128, (3, 3), padding='same', activation='relu')(concatenate_1)
conv7_5x5 = Conv2D(64, (5, 5), padding='same', activation='relu')(concatenate_1)
conv8_1x1 = Conv2D(64, (1, 1), padding='same', activation='relu')(concatenate_1)

# Concatenate the output of the convolutional layers above
concatenate_2 = concatenate([inception_3b_1x1, conv6_3x3, conv7_5x5, conv8_1x1], axis=3)

# Add a fully connected layer with 1024 neurons
model.add(layers.Dense(1024, activation='relu'))

# Add a dropout layer to prevent overfitting
model.add(layers.Dropout(0.4))

# Add the final output layer with 10 neurons (one for each class)
model.add(layers.Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])




The proposed architecture is inspired by GoogLeNet, which is a well-known deep learning architecture used for image classification tasks. In the case of Arabic sign language classification, the proposed model takes grayscale images of size 64x64 pixels as input and passes them through a series of convolutional and pooling layers to extract features at different scales. The added layers in this model help improve its accuracy and enhance the ability of the model to capture complex patterns in the input images.

In particular, the inception modules in the proposed architecture are designed to capture both local and global features of the input images, which is essential for recognizing signs in Arabic sign language. Additionally, the added dropout layer helps prevent overfitting by randomly dropping out some of the neurons during training, which improves the model's generalization ability.

Overall, the proposed architecture is well-suited for the classification task of Arabic sign language because it can effectively capture the intricate hand gestures and motions that are involved in this language.
